In [7]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings,ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
from langchain import hub
from langchain.agents import create_openai_tools_agent
from langchain.agents import AgentExecutor
import os

from dotenv import load_dotenv

load_dotenv()


os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")


In [5]:
# Create an instance of the WikipediaAPIWrapper class with the top result and a maximum of 200 characters for each document
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)

# Create an instance of the WikipediaQueryRun class using the WikipediaAPIWrapper instance
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

# Create a WebBaseLoader instance with the URL of the website to be loaded
loader = WebBaseLoader("https://microsoft.github.io/autogen/")

# Load the documents from the specified website
docs = loader.load()

# Split the loaded documents into chunks of 1000 characters with an overlap of 200 characters
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

# Create a FAISS database from the split documents using OpenAIEmbeddings
vectordb = FAISS.from_documents(documents, OpenAIEmbeddings())

# Create a retriever from the FAISS database
retriever = vectordb.as_retriever()

# Display the retriever
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x127915160>)

In [10]:
# Tools for the agent to use

# wikipedia tool for the agent to use to search for information
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)


#  tool for the agent to use to search for information about Autogen
retriever_tool=create_retriever_tool(retriever,"Autogen_search",
                      "Search for information about Autogen. For any questions about Autogen, you must use this tool!")


# arxiv tool for the agent to use to search for information 
arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)


# Tool kit for the agent to use
tools =[wiki,retriever_tool,arxiv]

In [8]:
# create an llm instance and prompt template
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

# Default prompt can be changed to any prompt
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [11]:
# Create an agent with the tools and the llm instance
agent = create_openai_tools_agent(llm,tools,prompt)

# agent executor to run the agent
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [12]:
agent_executor.invoke({"input":"Tell me about Autogen."})




> Entering new AgentExecutor chain...

Invoking: `Autogen_search` with `{'query': 'Autogen'}`


AutoGen | AutoGen








Skip to main contentWhat's new in AutoGen? Read this blog for an overview of updatesAutoGenDocsGetting StartedInstallationTutorialUser GuideAPI ReferenceFAQsEcosystemContributor GuideResearchExamplesExamples by CategoryExamples by NotebookApplication GalleryOther LanguagesDotnetBlogGitHubDiscordTwitterctrlKAutoGenEnable Next-Gen Large Language Model ApplicationsGetting Started - 3min ⏱️Multi-Agent Conversation FrameworkAutoGen provides multi-agent conversation framework as a high-level abstraction. With this framework, one can conveniently build LLM workflows.Easily Build Diverse ApplicationsAutoGen offers a collection of working systems spanning a wide range of applications from various domains and complexities.Enhanced LLM Inference & OptimizationAutoGen supports enhanced LLM inference APIs, which can be used to improve inference performance and reduce cost.Comm

{'input': 'Tell me about Autogen.',
 'output': 'Autogen is a platform that provides a multi-agent conversation framework as a high-level abstraction. It allows users to conveniently build workflows for large language models (LLMs). Autogen offers a collection of working systems spanning various applications from different domains and complexities. Additionally, Autogen supports enhanced LLM inference APIs to improve inference performance and reduce costs. For more information, you can visit the AutoGen website or check out their documentation, tutorials, and examples.'}

In [13]:
agent_executor.invoke({"input":"What's the paper 2403.19887 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '2403.19887'}`


Published: 2024-03-28
Title: Jamba: A Hybrid Transformer-Mamba Language Model
Authors: Opher Lieber, Barak Lenz, Hofit Bata, Gal Cohen, Jhonathan Osin, Itay Dalmedigos, Erez Safahi, Shaked Meirom, YonThe paper with ID 2403.19887 is titled "Jamba: A Hybrid Transformer-Mamba Language Model." It is authored by Opher Lieber, Barak Lenz, Hofit Bata, Gal Cohen, Jhonathan Osin, Itay Dalmedigos, Erez Safahi, Shaked Meirom, and Yon. The paper discusses a hybrid transformer-mamba language model called Jamba.

> Finished chain.


{'input': "What's the paper 2403.19887 about?",
 'output': 'The paper with ID 2403.19887 is titled "Jamba: A Hybrid Transformer-Mamba Language Model." It is authored by Opher Lieber, Barak Lenz, Hofit Bata, Gal Cohen, Jhonathan Osin, Itay Dalmedigos, Erez Safahi, Shaked Meirom, and Yon. The paper discusses a hybrid transformer-mamba language model called Jamba.'}